#Detection of Trending Repositories Communities
-- Shaily Patel / MIE1512

##1. Introduction
Over time, the number of people becoming a part of the online social network increased rapidly. There’s no denial that social media has transformed the way we interact with each other right from sharing our thoughts and photos to accessing interesting information streams created by relevant users. Most people tend to organise their social lives, or at least have it significantly influence them through some form of technology-based engagement. In order to enhance personalization, social recommender systems are in charge of filtering content streams based on each user’s interests model, their trusted social connections activity, and content authority. To do this, one way of  finding relevant items to recommend to a user would be to discover their meaningful connections. For instance, the degree of significance could be measured in terms of the impact of the resources the user shares and the links the user has with those inside a topic-dependent community. In order to tackle the novel problem of detecting communities related to a trending topic, the paper I selected,'**Detection of Trending Topic Communities: Bridging Content Creators and Distributors**', focuses on Twitter data and the **TreToC** approach described by the authors, empowers the desired spread of information. Their method identifies groups of topic-dependent “content creators" (CCs) in Twitter. Another key element of their proposal is the identification of their matching spreader groups or topic-dependent “content distributors" (CDs). After the identification of these two categories of users, both CCs and CDs are linked by the approach in a unique community, which represents the user base for the different forms of recommendation. 

The motive of my research is to implement this approach on the Github datasets with the needed modifications. The objective is to identify users that are interested on a similar subject and based on it observe a relevant user group. Individual users inside a community might receive recommendations of content generated by the other users, or the community as a whole could receive group recommendations, with new content related to that trending repository. This report includes methods that identifies topic-dependent user groups by linking those who generate the content (repository contributors) and those who spread this content(watchers). 

***Note: In the project, the term content creators(CC) refers to the contributors of the repository and content distributors(CD) refers to the watchers.***

###Importing libraries

In [4]:
from pyspark.sql import SparkSession
import networkx as nx
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame
from community import community_louvain
import matplotlib.pyplot as plt
from functools import reduce
from pyspark.sql.types import *
sql_sc = SQLContext(sc)


***Note : As we are using databricks, you will have to manually import the libraries***

You can do so by following the steps below:

1.Go to **Workspace**, when the tab opens, select the down arrow **(⌄)**.

2.Go to **Create** -> **Library**.

3.In the **Source** select **Upload Python Egg or PyPI**

4.In the **PyPi Name:** first write : **networkx==2.1**, then click **Install Library**.

5.Repeat steps 1-3 and in **PyPi Name:**, write **python-louvain**, then click **Install Library**.

###Importing files

* The following command imports the table ***project_members*** which has been extracted from GHTorrent itself.

In [8]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/157watcher/contributors_full.csv


* The following wget command imports a file **watcherMain**. This file was generated in GHTorrent using the following query(Query #1). As the watchers table was too large, I couldnt import the entire table so fetched the repositories that had watchers' count between 800-1000.

Query #1 
```sql
Select repo_id, user_id
From
[ghtorrent-bq:ght_2018_04_01.watchers]
where repo_id in
(select repo_id from (SELECT repo_id, count(user_id) as wat 
FROM [ghtorrent-bq:ght_2018_04_01.watchers] 
group by repo_id  having wat < 1000 and wat > 800))
```

In [10]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/157watcher/watcherMain.csv


* The following command imports the table ***followers*** which has been extracted from GHTorrent itself.

In [12]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/157watcher/followers1.csv


###Defining Schemas

In [14]:
dfwatchersSchema =  StructType([StructField('repo_id',StringType(),True),
                              StructField('user_id',StringType(),True)])
dfcontributorsSchema = StructType([StructField('repo_id',StringType(),True),
                                  StructField('user_id',StringType(),True),
                                  StructField('created_at',TimestampType(),True)])
dffollwersSchema = StructType([StructField('user_id',StringType(),True),
                              StructField('follower_id',StringType(),True),
                              StructField('created_at',TimestampType(),True)])


* Loading them into spark :

In [16]:
dfwatchers = spark.read.csv("file:/FileStore/tables/watcherMain.csv",header=True,schema=dfwatchersSchema)
dfcontributors = spark.read.csv("file:/FileStore/tables/contributors_full.csv",header=True,schema=dfcontributorsSchema)
dffollowers = spark.read.csv("file:/FileStore/tables/followers1.csv",header=True,schema=dffollwersSchema)

* Creating a new temporary view using a SparkDataFrame in the Spark Session.

In [18]:
dffollowers.createOrReplaceTempView("dffollowers")
dfcontributors.createOrReplaceTempView("dfcontributors")
dfwatchers.createOrReplaceTempView("dfwatchers")

##2. Questions related to Data Wrangling

###2.1 Data structure access questions:

1. Do all records in the dataset contain the same fields ?

Answer : I am using 3 dataframes for this analysis. The dataframe dfcontributors is directly imported from GHTorrent and have fields like repo_id , user_id , created_at , dont_use. The other dataframe dfwatchers was too large to be imported so I used a query to extract repositories and their watchers. I took into consideration only those repositories that have watchers in the range of 800-1000(reason mentioned later in the notebook) and extracted the table. This table has fields like repo_id and user_id. The last dataframe used, dffollowers, is a table directly extracted from GHTorrent. It has fields like user_id , follower_id and created_at that contains id of a user and the second field contains id of its follower. 

So the dataset do have **two common fields** like repo_id and user_id that I shall be using for other queries.

In [22]:
dfcontributors.show()

In [23]:
dfwatchers.show()

In [24]:
dffollowers.show()

2.How can you access the same fields across records? By position? By name? 

Answer : I accessed the same fields using the name of the dataframe followed by the respective field I wanted to use. I used the join operator whenever their was a  need to merge or blend the records together based on shared fields.

###2.2 Questions regarding data granularity:

1.What kind of thing (person, object, relationship, event, etc.) do the records represent? 

Answer : As we are using GitHub dataset, most of the data is related to repositories, contributors, organizations, forkers, watchers, etc. The dataset I used in my analysis is:

i) **dfcontributors**: In this data set, we have a repository's id for example 'x' and a user's id for example 'y'. This shows that the user 'y' is a contributor of the repository 'x'. The created_at field represents the time and data the user 'y' was added as a contributor in that repositpry 'x'.

ii) **dfwatchers**: In this data set, we have a repository's id for example 'x' and a user's id for example 'y'. This shows that the user 'y' watches the repository 'x'. 

iii) **dffollowers**: In this data set, we have a user's id for example 'x' and a follower's id for example 'y'. This shows that the user 'y' follows user 'x'. The created_at field represents the time and data when the user 'y' started following user 'x'. It shows a following relationship between two users.

2.Are the records homogeneous (represent the same kinds of things)? Or heterogeneous?

Answer: All the records in my dataset are homogeneous.

###2.3 Questions regarding Data Accuracy:

GHTorrent date-time accuracy :

The time component on dataset was troublesome in original form. Google BigQuery originally save timestamp in a certain format which is not recognised with spark SQL directly.

<i>open issue on github: https://github.com/GoogleCloudPlatform/google-cloud-go/issues/942 </i>

However,  a BigQuery function converts the datetime format to spark recognizable format. For example, to acquire the `followers` table, the following query was required:

```sql
SELECT  
 user_id, 
 follower_id, 
 STRFTIME_UTC_USEC(created_at, '%Y-%m-%d %H:%M:%S') AS created_at  
FROM [ghtorrent-bq:ght_2018_04_01.followers]```

*Source : Mohammad  (teammate)*

1.For date times, are time zones included or adjusted into a standard time zone like UTC? If the times are presented in 12-hour format, is AM/PM demarcated? Are the positions of month and day fields ambiguous (e.g., 01/02/17 versus 02/01/17)? Are there signs that the information is wrong (e.g., across your customer dataset, do you have an unusual amount of people with date-of-births corresponding to the first year of a decade like 1970, 1980, 1990, or born in January or on the first date of the month)?

Answer: The time zone of the dataset used is UTC. The times are presented in a 24-hour format so there is no issue of AM/PM. Also the positions of month and day are unambiguous. The date is already in the year-month-day format. 

The following query displays the date times so you can view the format of some sample records.

In [31]:
%sql
Select created_at from dfcontributors
Limit 5

2.Is there any chance of sensor drift that has caused systematic inaccuracies over time?

Answer: Sometimes malfunctions in the mirroring system (software or network) might result in some parts of the data that are missing. GHTorrent can restores the missing data by replaying the event log or using the ght-retrieve-repo script from Github. But, Github API does not permit retrieving more than the 300 newest events per repository. On busy projects, this is less than a day’s worth of event log. [2]

3.Is the data entered by people? If so, there might be a high incidence of mis-spellings and nonstandard abbreviations.

Answer : No. The data is collected from GHTorrent that gathers event streams and data from the GitHub hosting site and provides those data back to the community in the form of incremental MongoDB data dumps distributed through the peer-to-peer BitTorrent protocol.

###2.4 Questions regarding Data Temporality:

1.When was the dataset collected?

Answer : The data I am using in the project is collected from GHTorrent. This is from the latest dump titled 'ghtorrent-bq:ght_2018_04_01'. So we can say that the dataset has been collected on or before 1st April 2018 and has all the records till 1st April 2018. 

In the following command I queried all the data that was created after 1st April 2018 and the output was 'OK', meaning no event after that date is recorded.

In [36]:
%sql
Select * from dffollowers
having created_at > cast("2018-04-01T00:00:00.000+0000"as Timestamp)

2.Were all the records and record fields collected/measured at the same time? If not, is the temporal range significant?

Answer: 
>The created_at field is only filled in accurately for memberships for which GHTorrent has recorded a corresponding event. Otherwise, it is filled in with the latest date that the corresponding user or project has been created.
- source : http://ghtorrent.org/relational.html

So the data we have been assembled mostly in real time but, sometimes when they do not find the event they add the latest date when the corresponding user/project has been created.

The records are added in a dataset when the event happens. They might not neccesarily occur at the same time. For example, in the followers dataset when a user 'X' follows another user 'Y', the data is entered in the following table as : 'X' under follower_id and 'Y' under user_id and the date nd time of this event under created_at. There might be another following activity for different users at the same time. So all the events until 1st April 2018 have been recorded with the time stamp of when they occur. The different time of collection , i.e. the temporal range, is not significant when we have to use the data for the analysis. 

The below query displays the total number of events recorded at a same time. I have displayed the number of events taking place at ten different time.

In [38]:
%sql
SELECT created_at , count(*) as NUM FROM dffollowers GROUP BY created_at order by created_at  limit 10


Using the following query you can also display all the events that occured at a same time. The query displays all the events having the created_at field like '2009-11-22T22:57:13.000+0000'.

In [40]:
%sql
SELECT * FROM dffollowers where created_at like cast("2009-11-22T22:57:13.000+0000"as Timestamp)


3.Are the timestamps associated with collection of the data known and available (as a record field) or as associated metadata?

Answer : We do not have the timestamp of when the data is collected i.e. date and time of when the record is added in the dataset is unavailable. What we have is the timestamp of when the event described in the record occured.

The time and date of all the events are recorded with the data. For example you can find out the time and date when a user become a contributor of a certain repository. 

The following query retrives the timestamp of the events in the dataset dfcontributors. It returns details about every contributor of a certain repository and when they become a part of contributors. For example I have taken a repository with id = 108.

In [42]:
%sql
Select * from dfcontributors where repo_id = 108

4.Have some records or record field values been modified after the time of creation? Are the timestamps of these modifications available?

Answer : GHTorrent performs all mirroring operations twice by running instances of each specific crawler in two separate data centers, to ensure that all data generated by GitHub is accurately persisted in the system’s data store.The retrieved data are stored in a document-oriented NoSQL database, to accommodate future changes to the data returned by GitHub. The project details retrieval agent can setup a queue that collects all watch and fork events to query GitHub and update the project’s state in the local database mirror. Thus modifications do take place after the time of creation though we do not have the timestamps of these modifications.

Example of when a record might be modified is when a user is a watcher of a certain repository but may unwatch it after some time. Thus, this get updated when watch events are queried from the Github for a project and they no longer exist in the latest watchers table. 

The dataset used for this project contains all the modified events recorded till 1st April 2018. Changes made after this are not recorded.

5.In what ways can you determine if the data is “stale”? 

Answer: One can always use the Github's REST API and retrive the data s/he wants to verify. You can use pygithub libraries to manually check the authenticity of the data. 

For example you want to check whether user1 follows user2:
- You can use commands like "get_user(user2).has_in_following(get_user(user1))"

This returns a boolean value and one can verify the result with the dataset retrived from GHTorrent.

###2.5 Questions regarding Data Scope:

1.Given the granularity of the dataset, what characteristics of the things (e.g., people, objects, relationship, events, etc.) represented by the records are captured by the record fields? What characteristics are not captured?

Answer : Given that the granularity of the data is fine, we do not have multiple interpretation of the data in all the three datasets we use. For example in the following dataset, there is a record in the dataset when a user follows another user and the time when the following activity took place. We have no more details except these in the dataset. 

As for the characteristics, if we take the table dfcontributor, it just has data of a user's id, the repository's id in which the user is a contributor and the time and date the user become contributor. Other details of the user like user's country, forks made by the user, whether he is the owner of the repository or his pull_request was accepted is not present.

The query below displays all the characteristics a record has.

In [47]:
%sql
Select * from dfcontributors limit 5

2.Are the record fields consistent?

Answer : GHTorrent allows accommodation of future changes in the data when returned by GitHub. But we are mostly unaware of such changes as it incorporates the latest data and so we can not compare it with the previous version. For example if a user no longer follows a user it used too, you might not see the following relationship that existed before. Although if you have the data upto a certain time period, in our case 1st April 2018, the changes after that date would not be reflected in our dataset.

3.Are the same record fields available for all records? Are they accessible via the same specification ?

Answer : The id of users can be accessed via user_id, id of the repositories can be accessed via repo_id, the follower's id is accessed via follower_id and the time and date of the events that took place are accessed via created_at. These field names are consistent throughout all the datasets used in the project.

The folowing queries prints the schema of the dataframe along with the field names from which we shall access the records.

In [50]:
dffollowers.printSchema()

In [51]:
dfcontributors.printSchema()


In [52]:
dfwatchers.printSchema()

4.Are there missing records? 

Answer : As mentioned in one of the above questions, sometimes malfunctions in the mirroring system (software or network) might result in some parts of the data that are missing. Although it tries to restore, GHTorrent can not retrive more than 300 newest missing events per repository. [2]

##3. Data preparation

The analyses are performed on a dataset specifically built to collect information about selected repositories from Github. The dataset used for project comprises of 24 repositories. We need to have the following relationship between the content creators(contributors) , and the following relationship between the content distributors(watchers). This is collected by querying the GHTorrent. We have set a specific range for the number of watchers and contributors a repository should have to find the repositories. Github provides a list of trending repositories on its website but, there is no way to fetch that list or the data related to that list through the API. The final dataset contains 24 repositories, 776 unique creators, and 21012 unique distributors.

* The query below lists the repository id and the number of watchers and contributors it has. It selects the repositories that have watchers in the range of 800-1000 and contributors in the range of 20-50. I am using the repositories that resulted as the output of this query for my analysis. There are 24 repositories that fall into this category, which corresponds to the number of trending repositories GitHub has on its page. 

* Changing the number of watchers as well as contributors can give you different results. 

* Below is the query for getting the repositories that fall in your desired range. 

* ***Note :As I mentioned earlier, there is already a list of trending repositories available on GitHub, but due to the limitations in the API, I am unable to fetch those repositories and their data. So the repositories you select does not have much affect on this analysis. We are more focused to find communities, rather than selecting the repositories.***

In [57]:
%sql

Select w.repo_id as repo_id , w.wat as watcher, c.con as contributor from (SELECT repo_id, count(user_id) as wat 
FROM dfwatchers 
group by repo_id  having wat < 1000 and wat > 800) as w 
join 
(SELECT repo_id, count(user_id) as con 
FROM dfcontributors 
group by repo_id  having con < 50 and con > 20) as c on w.repo_id = c.repo_id 


#####How to select range for the above query

I used binning to see the distribution of watchers across all the repositories. As the watchers file is too large, I am unable to export it over here and run the   query. 
So, I ran the following query on GHTorrent
```sql
Select floor(wat/100.00)*100 as watchers, count(*) as count from 
(SELECT repo_id, count(user_id) as wat 
FROM [ghtorrent-bq:ght_2018_04_01.watchers]
group by repo_id ) as w 
group by watchers order by watchers
```
and imported the output. The distribution is as follows(Cmd #63) :

In [60]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/157watcher/binWatchers.csv

In [61]:
binWatchers = spark.read.csv("file:/FileStore/tables/binWatchers.csv",header=True)

In [62]:
binWatchers.createOrReplaceTempView("binWatchers")

In [63]:
%sql
Select watchers, count as repo_count from binWatchers having watchers > 100 and watchers < 1500 order by `watchers`+0 asc 

As we go down, the distribution is forming a tail. Also as we are considering trending repositories so it is obvious that such repositories shall have more watchers. As of now there is no option available in the GitHub API from which you can directly extract information about the trending repositories and the data related to them like contributors, watchers, etc. In the future application of this project, one may directly get list of trending reporitories(assuming it will be availble from GitHub API in the future) and form communities amoung them. I randomly selected the repositories that have watchers in the range of 800-1000. I selected this range as I think after 1000 watchers (bin) the number of repositories that can be selected are reducing. Selecting 800-1000 allows me to select from 2883 repositories. 

Now, for selecting the contributors' range I ran a similar query which I have mentioned below. I found the similar results that there are few repositories that have  more than 150 contributors. Below is the distribution of the number of repositories for every contributors' range. So I decided to select a random range of contributors below 150 and choose 20-50.

In [65]:
%sql
Select floor(con/10.00)*10 as contributors, count(*) as repo_id from 
(SELECT repo_id, count(user_id) as con 
FROM dfcontributors
group by repo_id ) as w 
group by contributors having contributors > 10 and contributors < 300 order by contributors


###3.1 Creation of Edges

Now you can randomly select any or all of the repositories as you like for creating a graph. 
* Each repository shall be represented as a graph.
* I selected all the 24 repositories and stored their id in a list ***'repoId'*** 
* Next I added a for loop that takes one repository id and create a graph for that repository. So this way I can create twenty-four graphs. 
* In every graph, the set of vertices is the union of contributors and watchers for the repository.
* The set of edges shall be the following relationship between two contributors , two watchers and also between a contributor and a watcher when a watcher follows a contributor of that repository.

A list of the 3 dataframes that is being created in the block(Cmd #79) and their description is mentioned in this block. You can see the queries that are used to create the dataframes below.
* ***contriEdges*** -  This dataframe is created by performing JOIN operation over dfcontributors and dffollowers. This query returns a row when a contributor is                          followed by another contributor of the same repository. So the dataframe basically represents **following relationship** between two                                contributors of a same repository and the dataframe that is returned shall be a part of the edge set between two nodes in the graph.
* ***watcherEdges*** - This dataframe is created by performing JOIN operation over dfwatchers and dffollowers. This query returns a row when a watcher is                                  followed by another watcher of the same repository. So the dataframe basically represents **following relationship** between two                                    watchers of a same repository and the dataframe that is returned shall be a part of the edge set between two nodes in the graph.
* ***contriWatcherEdges*** - This dataframe is created by performing JOIN operation over dfcontributors, dfwatchers and dffollowers. This query returns a row when a                              contributor is followed by a watcher of the same repository. So the dataframe basically represents **following relationship**                              between a contributor and a watcher and the dataframe that is returned shall be a part of the edge set between two nodes in the graph.

The following function ***unionAll*** takes as parameter different dataframes and returns a giant union dataframe.

- *Source : https://stackoverflow.com/questions/33743978/spark-union-of-multiple-rdds*

In [70]:
def unionAll(*x):
  return reduce(DataFrame.unionAll,x)

As mentioned above, in order to build the set E of edges that represent the connections among the users, we consider three types of relationships. 

The first is the *following* relationship between two repository-dependent content creators i.e. create an edge when a contributor of a certain repository follows another contributor of the same repository. Let's see how this is done for certain repository say repository_id = 3452. The output of the query would be the nodes between which we shall create an edge in the graph.

***Note: This is the sample of edge creation for all the three relationships for repository id : 3542***

In [72]:
%sql
 SELECT * FROM 

  (SELECT K.USER_ID as CONTRIBUTOR_node1, U.FOLLOWER_ID AS FOLLOWER_node2 FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfcontributors WHERE repo_id = 3452 ) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  WHERE FOLLOWER_node2 IN 

  (SELECT K.USER_ID as CONTRIBUTOR_node1 FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfcontributors WHERE repo_id = 3452) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  ORDER BY CONTRIBUTOR_node1

The second type of connection we consider is the *following* relationship between two topic-dependent content distributors i.e. when a watcher of a certain repository follows another watcher of the same repository. The output of the query would be the nodes between which we shall create an edge in the graph.

In [74]:
%sql
  SELECT * FROM 

  (SELECT  K.USER_ID as WATCHER_node1 , U.FOLLOWER_ID as FOLLOWER_node2 FROM
  (SELECT  F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfwatchers WHERE repo_id = 3452 ) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  WHERE FOLLOWER_node2 IN 

  (SELECT K.USER_ID as WATCHER_node1 FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfwatchers WHERE repo_id = 3452 ) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  ORDER BY WATCHER_node1



In the third type of connection we link a Content creator to a Content distributor only if CD follows a CC of that repository. The following table returns the nodes of all the edges to be created as a result of this relationship

In [76]:
%sql
  SELECT * FROM 

  (SELECT K.USER_ID as CONTRIBUTOR_node1, U.FOLLOWER_ID as FOLLOWER_node2 FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfcontributors WHERE repo_id = 3452 ) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  WHERE FOLLOWER_node2 IN 

  (SELECT K.USER_ID as WATCHER_node2 FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfwatchers WHERE repo_id = 3452 ) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)
  
  ORDER BY CONTRIBUTOR_node1
  
/*Source : Jay Patel (Teammate)*/
  

Now you can select the total number of repositories you want to use for your analysis and append their id to the list 'repoId'. This block(Cmd #79) builds all the 3 edge relationships and the vertices for each repository in the list 'repoId' and creates a graph respectively. This graph is then stored in a **Graph Exchange XML Format(gexf)** which we shall retrive later while creating communities. We are using the library 'networkx' for creating the graph. This takes an average of **70 minutes** for a list consisting of **24 repositories**.

###3.2 Steps for creating the graph:
1. Create dataframes to represent the 3 *following* relationships and find nodes to create edges.
2. Create a graph.
3. Add nodes to the graph.
4. Add edges collected from the dataframes to the graph.
5. Save the graph in a  *.gexf* file

In [79]:
users_id = [] 
repoId =[3452,44045790,11632,12104235,9952400,11697421,6359,22338,11660171,13294,2869,11645112,12333,8379,2202,7640,8131,12639,9663166,2496321,157546,6163,1597341,64336]
for i in repoId:
  di = {}
  
  #Resulting DataFrame shall be used for creation of edge between two contributors for a certain repository
  contriEdges = spark.sql(
  '''
  SELECT * FROM 

  (SELECT K.USER_ID as CONTRIBUTOR, U.FOLLOWER_ID AS FOLLOWER FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfcontributors WHERE repo_id = {0} ) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  WHERE FOLLOWER IN 

  (SELECT K.USER_ID as CONTRIBUTOR FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfcontributors WHERE repo_id = {1}) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  ORDER BY CONTRIBUTOR
  '''.format(i,i)
  ) 
  
  #Resulting DataFrame shall be used for creation of edge between two watchers for a certain repository 
  watcherEdges = spark.sql(
  '''
  SELECT * FROM 

  (SELECT  K.USER_ID as WATCHER, U.FOLLOWER_ID as FOLLOWER FROM
  (SELECT  F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfwatchers WHERE repo_id = {0} ) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  WHERE FOLLOWER IN 

  (SELECT K.USER_ID as WATCHER FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfwatchers WHERE repo_id = {1}) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  ORDER BY WATCHER

  '''.format(i,i)
  )
  
  #Resulting DataFrame shall be used for creation of edge between one contributor and one watcher for a certain repository 
  contriWatcherEdges = spark.sql (
  '''
  SELECT * FROM 

  (SELECT K.USER_ID as CONTRIBUTOR, U.FOLLOWER_ID as FOLLOWER FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfcontributors WHERE repo_id = {0}) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)

  WHERE FOLLOWER IN 

  (SELECT K.USER_ID as WATCHER FROM
  (SELECT F.user_id, COUNT(F.follower_id) FROM dffollowers as F 
  JOIN  (SELECT * from dfwatchers WHERE repo_id = {1}) as W ON W.user_id = F.user_id
  GROUP BY F.USER_ID  ORDER BY F.USER_ID) AS K
  JOIN dffollowers AS U ON K.USER_ID = U.USER_ID
  ORDER BY K.USER_ID)
  
  ORDER BY CONTRIBUTOR
  '''.format(i,i)
  )
  
  #Selection of nodes for the graph
  cr = spark.sql('''
    SELECT user_id from dfcontributors where repo_id = {0}
    '''.format(i)
                )
  dr = spark.sql('''
    SELECT user_id from dfwatchers where repo_id = {0}
    '''.format(i)
                )
  
  #Need to use the collect() method to collect data of the dataframe as Networkx needs list of users to create nodes.
  di['creators'] = cr.rdd.map(lambda p: str(p[0])).collect() 
  di['distributors']=dr.rdd.map(lambda p: str(p[0])).collect()
  di['topic']= str(i)
  
  edges = unionAll(contriEdges,watcherEdges,contriWatcherEdges)
  
  #Create a Graph
  G = nx.DiGraph()
  
  #Creating a list of edges for the graph.
  edgeList = edges.rdd.map(lambda p: (str(p[0]),str(p[1]))).collect()
  
  #Adding Edges to the graph.
  G.add_edges_from(edgeList)
  nx.write_gexf(G,'repo'+str(i)+'.gexf')
  users_id.append(di)
  G.clear()

The method **Directed_Clustering_Coefficient2** takes as a parameter a graph G that we previously created and returns the clustering coefficient. The community clustering coefficient, quantifies the extent to which nodes in the graph tend to cluster together. The clustering coefficient for nodes in a directed graph is defined by the method Directed_Clustering_Coefficient2. There is a method  **average_clustering(G, nodes=None, weight=None, count_zeros=True)** available in the networkx library, but this method is for an undirected graph and we need to find the clustering coefficient for the directed graph we created. Hence, the method described below.
- *Source : https://github.com/lore10/Detection-of-Trending-Topic-Communities_Datasets-Code*

In [81]:
def Directed_Clustering_Coefficient2(G): #Definition of function that takes input a directed graph  ##VALIDADO
    if nx.number_of_edges(G)>0: #Calculating the number of nodes in the graph
        p = nx.adjacency_matrix(G) #Finding the adjacency matrix of G
        adjacency_matrix = p.toarray()   
        clustercoeff = [] #Initializing a list for storing clustering coefficient values
        for node in G.nodes(): #Calculating clustering coefficient for each node in the graph
            local = 0
            #nb = G.neighbors(node) #Finds the neighbours of node, neighbours are the succesors of a node in a directed graph
            nb = list(G.predecessors(node))+list(G.neighbors(node))
            k = len(nb) #Finding the length of the neighbours array of node
            for n_node1 in nb: 
                for n_node2 in nb:
                    if adjacency_matrix[list(G.nodes()).index(n_node1)][list(G.nodes()).index(n_node2)] == 1 and  n_node1 != n_node2:
                        local = local + 1

            if(k>1): #Clustering Coefficient is only defined for nodes which have more than 1 out degrees
                node_ccoeff = float("{0:.3f}".format(local/(k * (k-1))))
                clustercoeff.append(node_ccoeff) #Storing clustering coefficient value for node i in list
            else: #Storing a value of 0 for those nodes who have out degree of 1 or 0
                clustercoeff.append(0)
    
        avg_cc= sum(clustercoeff)/len(clustercoeff)
        return avg_cc #Returns the clustering coefficient of all nodes in the network
    else:
        return 1000000

###4. Detection of Trending Repository Communities

The next step is to create communities. To build and manipulate the graph, as well as to calculate the metrics used for analysis, we used the NetworkX module. However, the clustering coeffecient of nodes for directed graphs is not part of the functions as we have defined the method above. To run the Louvain community detection algorithm and measure the graph modularity we used the community module. To define the communities, the function employed was community.best_partition() where the resolution parameter is set to 1. The resolution in modularity is used to adjust the optimization in partitioning. If this value is bigger than 1 it leads to the merging of two communities that share one or more edges, independently of the communities’ features. We did not alter the resolution to avoid bias. Because of the properties of Louvain, the directed graph needs to be transformed into undirected when calling the function.
- *Source : https://github.com/lore10/Detection-of-Trending-Topic-Communities_Datasets-Code*

To validate our proposal, four sets of analyses are performed:
  
(1) ***Characterization of the trending repositories***: Given a trending repository, we analyze the number of contributors and watchers that characterize it. This will allow us to understand the dynamics that characterize the activity on Github, even before communities are detected.

(2) ***Analysis of the disconnected users***: In this case, we analyze the percentage of disconnected users from the graph (which would not be involved in the community detection and thus would not benefit of the information spreading).

(3) ***Analysis of the cohesion among the users***: For each community, we evaluate its quality by measuring the cohesion between the users in it, using standard metrics such as modularity, ratio between the number of communities and the number of users, and density.

(4) ***Analysis of the community structure***: For each community, we analyze its composition, by measuring the ratio of content creators and distributors in it, and their clustering coeffecient. This allows us to evaluate the effectiveness of our approach to connect those who generate the content to those who make use of it.

In [85]:
list1=[]
for i in repoId:
    G=nx.DiGraph()
    G= nx.read_gexf('repo'+str(i)+'.gexf')
    
    #Select the creators and distributors of the repository for which you are going to create communities.
    ce=[u["creators"] for u in users_id if u['topic']==str(i)][0]
    di=[u["distributors"] for u in users_id if u['topic']==str(i)][0]
    total_users=len(ce)+len(di)
    num_nodes=G.number_of_nodes()
    connected=float("{0:.3f}".format(num_nodes/total_users))
    isolated=float("{0:.3f}".format(1-connected))

    G_unD=G.to_undirected()

    if G.number_of_edges()>0:
        
        #Calculation for metrics
        
        partition = community_louvain.best_partition(G_unD)
        modularity=float("{0:.3f}".format(community_louvain.modularity(partition, G_unD)))
        num_communities = len(set(partition.values()))
        ratio_comm_nodes=float("{0:.3f}".format(num_communities/num_nodes))

        density=float("{0:.3f}".format(nx.density(G)))

        nodes=[n for n in G.nodes()]
        numCr_inGraph=len([creat for creat in ce if creat in nodes])
        numDi_inGraph=len([dist for dist in di if dist in nodes])
        ratio_cr_comm=float("{0:.3f}".format(numCr_inGraph/num_communities))
        ratio_di_comm=float("{0:.3f}".format(numDi_inGraph/num_communities))

        ccoeff=float("{0:.3f}".format(Directed_Clustering_Coefficient2(G)))

    else:
        modularity=0
        num_communities=0
        ratio_comm_nodes=0
        density=0
        ratio_cr_comm=0
        ratio_di_comm=0
        ccoeff=0

    list1.append([len(ce),len(di),total_users,num_nodes,connected,isolated,modularity,num_communities,ratio_comm_nodes,density,ratio_cr_comm,ratio_di_comm,ccoeff])
    print('The repository is :'+str(i)+'; '+str(len(ce))+';'+str(len(di))+';'+str(total_users) +';'+ str(num_nodes) +';'+ str(connected) +';',
               str(isolated) +';'+ str(modularity) +';'+ str(num_communities) +';'+ str(ratio_comm_nodes) +';',
               str(density) +';'+ str(ratio_cr_comm) +';'+ str(ratio_di_comm) +';'+ str(ccoeff))
    
    G.clear()

To find the average of the metrics of all the repositories I used the following code:
```python
numlist = []
for i in list1:
  numlist.append(i[])
x= sum(numlist)/len(numlist)
```
I used these average values to compare the analysis outputs with the results in the paper.

##5. Analytical Results

In the following cells, we provide a detailed evaluation of the proposal. Use of boxplot has been done to represent the analytical results.

The following query creates a boxplot for the list of output from the above cell (Cmd #85).

In [89]:
#Lists for creating bloxplot graph.
list2 = []
for i in range(13):
    list3 = []
    for j in list1:
        list3.append(j[i])
    list2.append(list3)

#### 5.1 Characterization of the trending repositories
The first boxplot represents the ***characterization of the trending repositories***. 

In the following, we analyze what characterizes the repositories in the dataset. The first boxplot shows average number of creators and distributors per repositories.

In [91]:
data_to_plot = [list2[0], list2[1]]

fig = plt.figure()
plt.title("Figure 1")
# Create an axes instance
ax = fig.add_subplot(111)

# Create the boxplotx
bp = ax.boxplot(data_to_plot)
plt.xticks([1, 2], ['contributors', 'watchers'])
ax.set_ylabel('Distribution')

# Save the figure
fig.savefig('fig11.png', bbox_inches='tight')
display(fig)

#####Graph interpretation of Figure 1
Each boxplot in Figure 1 represents the number of creators who contributed in the repositories and the number of distributors who watches those repositories. From the distributions obtained as results, we see that content generation and content propagation behave differently. 

The average number of creators per trending topic is 32.3 while the median number of distributors per trending topic is 865.0.

We can see that for every repository, the number of watchers are approximately 30 times more than the number of contributors. This may be the result of the range we selected at the first place for finding the repositories used in our analysis.

#### 5.2 Analysis of the disconnected users

When a user has a connection with another, it is going to be considered in a graph (as a source or destination node, depending on the relationship). Accordingly, the average number of disconnected users for the trending topics was analyzed. Non-linked users are detected once the repository graph is obtained, while the rest of the users shape the main connected component.

The results show that the repository graphs generated loses 49.11% of the user base of the dataset. When this method is used on Twitter data, it covers 85% of the users who take part in the trending topic for creating communities. 

These results demonstrate that this approach might not be the most efficient way to find communities among the Github data. The reason behind this may be that the contributors and watchers are not closely tied within themselves. (i.e. few following relationships among contributors and watchers)

#### 5.3 Analysis of the cohesion among the users
The next boxplot represents the ***Analysis of the cohesion among the users***. 

In order to analyze the level of cohesion between the users in a community, in Table 1 we report the average values of modularity, ratio between the number of communities and the number of users, and density.

The corresponding metric values obtained are presented in Figure 2. A lower modularity shows that the communities in the graph maintain certain level of interaction or connection between them. Furthermore, the density is influenced by the fact, that when the users share two links and act as source or destination nodes, it results in a bigger value of density.

Table 1

|Modularity| Ratio of Communities/Nodes | Density |
|----------|:--------------------------:|--------:|
|0.475     |                   0.0407 |   0.005 |

In [96]:
'''
Analyze the level of cohesion between the users in a community, and report the average values of modularity, ratio between the number of communities and the number of users, and density.
'''
cohesion = [list2[6], list2[8], list2[9]]
fig1 = plt.figure()
plt.title("Figure 2")

# Create an axes instance
ax1 = fig1.add_subplot(111)

# Create the boxplotx
bp1 = ax1.boxplot(cohesion)
plt.xticks([1, 2,3], ['Modularity', 'Ratio_comm_nodes','Density'])
ax1.set_ylabel('Values for each treanding topics')
# Save the figure
fig1.savefig('fig1.png', bbox_inches='tight')
display(fig1)

#####Graph interpretation of Figure 2
The boxplot(Figure 2) analyzes the level of cohesion between the users in a community, and reports the distribution  of modularity, ratio between the number of communities and the number of users, and density.

Purpose of this work is to get fewer communities, which are highly associated units composed by a suitable number of content creators and distributors. For example, more linked content creators in a community would cause diversity in future recommendations.

Compared to the modularity value for the Twitter data (for Twitter : 0.622), a lower modularity is obtained  for Github (for Github : 0.475). This shows that the communities in the graph maintain certain level of interaction or connection between them. 

On the contrary, the value of density for repository's communities(0.005) is very low compared to the density of graphs obtained from Twitter data(0.027). This may be due to the number of links nodes share in each graphs.

The average number of communities found in github's repository graphs is 16.79, that is almost similar to the average amount of communities found in Twitter's trending topics' graphs (16.22 communities per trending topic) i.e., for both the data the method approachs to obtain larger communities.

#### 5.3 Analysis of the community structure
The final boxplot represents the ***Analysis of the community structure***. 

This analysis measures the average percentage of content creators, the average percentage of content distributors, and the average clustering coefficient for a given community.

In [99]:
'''
a summary of the analysis of the community structure obtained for the set of repository used in my study. Have to analyize why did creators/community and distributors/community came up as zero
'''
community_structure = [list2[10], list2[11], list2[12]]
fig2 = plt.figure()
plt.title("Figure 3")

# Create an axes instance
ax2 = fig2.add_subplot(111)

# Create the boxplotx
bp2 = ax2.boxplot(community_structure)
plt.xticks([1, 2,3], ['contributors/community', 'watchers/community','avg CC'])    #Clustering Coefficient= CC
ax2.set_ylabel('Values for each treanding topics')
# Save the figure
fig2.savefig('fig2.png', bbox_inches='tight')
display(fig2)

#####Graph interpretation of Figure 3
In this method, the *following* relationship joins content creators making it more likely to  find them as close neighbors. Consequently, their individual distributors come together too. We can observe this in the percentage of content distributors in a community, which is bigger too.

As a consequence of being able to have more content creators and distributors linked together in a Github communities, the clustering coefficient is more compared to that of Twitter communities.

The average clustering coefficient for Github communities is 0.1286 and that for twitter communities is 0.077.

The boxplots in Figure 3, report the results of the three metrics found over the selected repositories.

From the results, we notice that using the TreToC method yeilds average results for creating communities in the GitHub as well.

##6. Conclusions

* In this work, we have applied a previously proposed framework to bring together repository-dependent contributors and watchers groups and identify relations between them. This approach is quite efficient for finding communities in Twitter and can be useful for recommender systems as well. 

* This approach identifies the proper links between the users who participate in the contribution of a repository and then to detect suitable communities, which contain both creators and distributors.

* From our analytic results we may conclude that this method may be applied for GitHub as it is making large-sized and meaningful modular communities. Although it leaves a larger user base (~50%) in forming communities. 

* Communities with only 50% of the users might not be that helpful for recommender systems, as you need spread of information to maximum possible users. 

* In this project we have used a single repository as a trending topic, but it might be more significant when you group repositories with similar characteristics and treat it as a single trending topic. Finding communities between them may yeild positive results as the possibilities of the users connected among each other would increase.

## 7. Future Work

* Finding the Jaccard index to measure the overlap of users among two repositories remains the future scope of this project. Jaccard index would let us know whether the users are gathered together because of the topic or because of previous relationships between them.

* Also we can compare the outputs of this project with some other community detection methods and see which gives better results.

##8. References:
1. Lorena Recalde, David F. Nettleton, Ricardo Baeza-Yates, and Ludovico Boratto. 2017. Detection of Trending Topic Communities: Bridging Content Creators and Distributors. In Proceedings of the 28th ACM Conference on Hypertext and Social Media (HT '17). ACM, New York, NY, USA, 205-213. DOI: https:// doi.org/10.1145/3078714.3078735

2. Georgios Gousios. 2013. The GHTorent dataset and tool suite. In Proceedings of the 10th Working Conference on Mining Software Repositories (MSR '13). IEEE Press, Piscataway, NJ, USA, 233-236.

3. Georgios Gousios and Diomidis Spinellis. 2012. GHTorrent: GitHub's data from a firehose. In Proceedings of the 9th IEEE Working Conference on Mining Software Repositories (MSR '12). IEEE Press, Piscataway, NJ, USA, 12-21.

4. V. D. Blondel, J. L. Guillaume, R. Lambiotte, and E. Lefebvre. 2008. Fast unfolding of communities in large networks. Journal of Statistical Mechanics: Theory and Experiment 2008, 10 (2008), P10008.

5. J. Weng, E. P. Lim, J. Jiang, and Q. He. 2010. TwitterRank: Finding Topic-sensitive Influential Twitterers. In Proceedings of the Third ACM International Conference on Web Search and Data Mining (WSDM ’10). ACM, New York, NY, USA, 261–270. https://doi.org/ 10.1145/1718487.1718520.

6. U. M. Dholakia, R. P. Bagozzi, and L. K. Pearo. 2004. A social influence model of consumer participation in network- and small- group-based virtual communities. International Journal of Research in Marketing 21, 3 (2004), 241 – 263.